In [11]:
!pip3 install llama_index nebula3-python futures glob2 pypdf sentence_transformers langchain ipython-ngql pyvis networkx

  Obtaining dependency information for ipython-ngql from https://files.pythonhosted.org/packages/4a/1a/11cfe058ae84df31f503d115941fe665b116ef6fee3488ba284d03ba45c3/ipython_ngql-0.7.5-py3-none-any.whl.metadata
  Using cached pyvis-0.3.2-py3-none-any.whl (756 kB)
  Obtaining dependency information for ipython>=5.3.0 from https://files.pythonhosted.org/packages/ef/02/fc039fca3ec40a00f962eb6e9da45c507334b0650a3cb9facd38d234fb7a/ipython-8.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpickle>=1.4.1 from https://files.pythonhosted.org/packages/d3/25/6e0a450430b7aa194b0f515f64820fc619314faa289458b7dfca4a026114/jsonpickle-3.0.2-py3-none-any.whl.metadata
  Using cached backcall-0.2.0-py2.py3-none-any.whl (11 kB)
  Using cached decorator-5.1.1-py3-none-any.whl (9.1 kB)
  Obtaining dependency information for jedi>=0.16 from https://files.pythonhosted.org/packages/20/9f/bc63f0f0737ad7a60800bfd472a4836661adae21f9c2535f3957b1e54ceb/jedi-0.19.1-py2.py3-none-any.whl.metadata

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [2]:
import os

os.environ["EDENAI_API_KEY"] = "*****.****.*****"
os.environ["OPENAI_API_KEY"] = "******************************"

In [3]:
from llama_index import KnowledgeGraphIndex, LLMPredictor, ServiceContext, utils
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

from langchain import OpenAI
from langchain.llms import EdenAI
from IPython.display import Markdown, display
from sentence_transformers import SentenceTransformer, util

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader("./docs").load_data()

In [5]:
print(len(documents))
documents[0]

17


Document(id_='97119cf4-965e-4014-97f9-541a690d75f1', embedding=None, metadata={'page_label': '1', 'file_name': 'P02-1058.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='900803dc9e1b40eb30f903a63058fa2dcb46e1f9b2e20d75f46245da695c2dfe', text='From Single to Multi -document Summarization:  \nA Prototype System and its Evaluation  \nChin-Yew Lin and Eduard Hovy  \nUniversity of Southern California / Information Sciences Institute  \n4676 Admiralty Way  \nMarina del Rey, CA 90292  \n{cyl,hovy}@isi.edu  \n \nAbstract  \nNeATS is a multi -document \nsummarization system that attempts \nto extract relevant or interesting \nportions from a set of documents \nabout some topic and present them \nin coherent order. NeATS is among \nthe best performers in the large scale \nsummarization evaluat ion DUC \n2001.  \n1 Introduction  \nIn recent years, text summarization has been \nenjoying a period of revival.  Two workshops \non Automatic Summarization w

In [6]:
connection = "--address 127.0.0.1 --port 9669 --user root --password nebula;"
space_name = "xyz"
%load_ext ngql

In [7]:
%ngql {connection}

Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,xyz


In [8]:
%ngql CREATE SPACE IF NOT EXISTS {space_name}(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [9]:
%ngql USE {space_name};
%ngql CREATE TAG IF NOT EXISTS entity(name string);
%ngql CREATE EDGE IF NOT EXISTS relationship(relationship string);

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [10]:
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [11]:
!pip3 install nltk
import nltk

nltk.download("stopwords")


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhinavvengala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from llama_index.embeddings import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

llm = EdenAI(provider="openai", model="text-davinci-003", temperature=0, max_tokens=512)
embeddings = LangchainEmbedding(HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

llm_predictor = LLMPredictor(llm=llm)
service_context = ServiceContext.from_defaults(
    llm=llm, chunk_size=512, embed_model=embeddings
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [14]:
import os

os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default password
os.environ["NEBULA_ADDRESS"] = (
    "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally
)

# space_name = "xyz"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omitted if created from an empty kg
tags = ["entity"]

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [15]:
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=5,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    include_embeddings=True,
    tags=tags,
)


(NeATS, is, multi-document summarization system)
(NeATS, attempts to extract, relevant or interesting portions)
(NeATS, is among, best performers in large scale summarization evaluation DUC 2001)
(DUC, sponsors, Text Summarization Challenge)
(NTCIR, started in, 2000 in Japan)


Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00,  5.30it/s]



(NeATS, is, multi-document summarization system)
(NeATS, leverages, techniques)
(NeATS, uses, term clustering)
(NeATS, generates summaries in, three stages)
(Content selection, goal is to, identify important concepts)


Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 28.47it/s]



(AA flight 11, is key concept for, September 11 terrorist attacks)
(AA flight 77, is key concept for, September 11 terrorist attacks)
(UA flight 173, is key concept for, September 11 terrorist attacks)
(UA flight 93, is key concept for, September 11 terrorist attacks)
(New York, is key concept for, September 11 terrorist attacks)


...

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 33.78it/s]



(James Shaw, published, Conciseness through aggregation in text generation)
(Gees C. Stein, Tomek Strzalkowski, G. Bowden Wise, published, Summarizing Multiple Documents Using Text Extraction and Interactive Clustering)
(Tomek Strzalkowski, Jin Wang, Bowden Wise, published, A robust practical text summarization system)
(J. I. Tait, published, Automatic Summarizing of English Texts)
(Simone Teufel, Marc Moens, published, Sentence extraction as a classification task)


Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 42.43it/s]



(page_label, is, 9)
(file_name, is, W00-0405.pdf)
(SIGIR'98, held in, 1998)
(Yiming Yang, et al., published, Learning approaches for topic detection and tracking news events)
(IEEE Intelligent Systems, published, Special Issue on Applications of Intelligent Information Retrieval)


Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 27.42it/s]


In [23]:
kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="keyword",
    verbose=True,
    response_mode="tree_summarize",
)

# Using KG as a Query Engine (QA)
response = kg_index_query_engine.query("problems in multi document summarization")

display(Markdown(f"<b>{response}</b>"))

Extracted keywords: ['multi', 'summarization', 'document', 'problems']
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: f7f5d96b-d7f9-4a57-9b7b-b058ecf6f42b: 1995. Lexical se- 
mantics in summarization. In Proceedings of the First 
Ann...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 797819d3-2f94-4924-8593-a9cc77aceb69: document summarization by using additional, available 
information about the ...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 3095c736-a667-4250-a0fa-34d033c9e339: i However, large- 
scale IR and summarization have not yet been truly in- 
te...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
summarization{name: summarization} <-[relationship:{relationship: has not been integrated with}]- IR{name: IR}
summarization{name: summarization} <-[relationship:{relationshi

<b> Multi-document summarization has several problems, such as selecting, evaluating, ordering, and aggregating items of information according to their relevance to a particular subject or purpose. Additionally, the system must take into account other summaries that have already been generated, and must be able to summarize either complete document sets or single documents in the context of previously summarized ones.</b>

In [151]:
from llama_index.retrievers import KGTableRetriever

# Using KG as an Information Retriever
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="hybrid", include_text=True
)
# retriever_mode
# keyword: Uses keywords to fetch
# embedding: Uses embeddings
# hybrid: Uses both

# include_text=True  (return the nodes as text without unnecessary characters)
# inlcude_text=False (returns the nodes as nodes and pointers)

In [152]:
nodes = kg_retriever.retrieve("NeATS")

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00,  8.84it/s]

INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 132dce5a-b0ac-49a8-995a-4137e17dc628: If we count the number of model units that are 
marked as good summary units ...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 5bb8f509-52e6-4fba-ac18-dda60bc94c7f: In a k ey step for locating important sentences, 
NeATS computes the likeliho...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 43710785-afef-4cfd-88e3-d365924a63e6: We described a multi -document 
summarization system, NeATS, and its 
evaluat...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 522dbc1d-6914-4cd9-9439-935c1fb5cbb3: In this paper we describe a multi -document 
summarization system NeATS.  It ...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: b3f70797-b8ab-40b0-aae3-9345f2bd9639: From Single to Multi -document Summarization:  
A Prototype System and its Ev...
INFO:llama_index.indices.knowledge_graph

In [153]:
print(len(nodes))
text = ""
for i in nodes[:-1]:
    text += i.text
text

9


'If we count the number of model units that are \nmarked as good summary units and are \nselected by systems, and use the number of \nmodel units in various summary lengths as the \nsample space, we obta in a precision metric \nequal to Retention 1.  Alternatively, we can \ncount how many unique system units share \ncontent with model units and use the total \nnumber of system units as the sample space.  \nWe define this as pseudo precision, Precision p, \nas follows:  \nsummary system in the SUs of number  Totalmarked SUs of Number  \nMost of the participants in DUC -2001 reported \ntheir pseudo precision figures.  \n5 Results and Discussion  \nWe present the performance of NeATS in \nDUC -2001 in content and quality measures.  \n5.1 Content  \nWith respect to content, we computed \nRetention 1, Retention w, and Precision p using \nthe formulas defined in the previous section.  \nThe scores are shown in Table 1 (overall \naverage and per size).  Analyzing all systems’ \nresults accord

In [154]:
from langchain import PromptTemplate, LLMChain

template = """
Write brief summary of {topic} by fully utilizing information:
```{nodes}```
"""
prompt = PromptTemplate(template=template, input_variables=["topic", "nodes"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
summary = llm_chain({"topic": "NeATS", "nodes": text})

In [155]:
display(Markdown(f"<b>{summary['text']}</b>"))

<b>

NeATS is a multi-document summarization system that attempts to extract relevant or interesting portions from a set of documents about some topic and present them in coherent order. It leverages techniques such as term frequency, sentence position, stigma words, and a simplified version of MMR to select and filter content. It also uses term clustering, a ‘buddy system’ of paired sentences, and explicit time annotation to improve topic coverage and readability. NeATS was evaluated in the Document Understanding Conference (DUC) 2001 and Text Summarization Challenge (TSC) 2000, and performed well in content and readability measures. It was consistently ranked among the top 3 in average and per size Retention 1 and Retention w, and its performance for averaged pseudo precision equals human’s at about 58%. However, the unweighted retention of humans is 53%, suggesting assessors did write something similar in their summaries but not exactly the same. NeATS is still a prototype system and can be improved with compression techniques, linguistic units smaller than sentences, discourse processing techniques, and improved and more stable metrics.</b>